In [ ]:
!pip install pandas folium matplotlib seaborn openpyxl papaparse pygame

ERROR: Could not find a version that satisfies the requirement papaparse (from versions: none)
ERROR: No matching distribution found for papaparse


In [23]:
# -*- coding: utf-8 -*-
"""
Зорилго: Замын хөдөлгөөний зөрчлийн географик тархалтын анализ
Хэрэглэх хувьсагчид: 
    - longitude_col: 'Уртраг'
    - latitude_col: 'Өргөрөг'
    - region_cols: ['Аймгийн нэр', 'Дүүргийн нэр', 'Сумын нэр', 'Хорооны дугаар']
    - road_cols: ['Авто зам - Замын хэсэг', 'Авто зам - Замын ангилал', 
                 'Авто зам - Замын харьяалал', 'Авто зам - Замын хучилт', 'Авто зам - Замын гадаргуу']
    - date_col: 'Зөрчил огноо'
"""

import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, HeatMap

# 1. Өгөгдөл унших, цэвэрлэх
file_path = 'ЗТО_2020-2024_ашиглах_final.xlsx'
df = pd.read_excel(file_path)

# Шаардлагатай багануудыг үлдээх
cols = ['Уртраг', 'Өргөрөг', 'Аймгийн нэр', 'Дүүргийн нэр', 
        'Сумын нэр', 'Хорооны дугаар',
        'Авто зам - Замын хэсэг', 'Авто зам - Замын ангилал', 
        'Авто зам - Замын харьяалал', 'Авто зам - Замын хучилт', 
        'Авто зам - Замын гадаргуу', 'Зөрчил огноо']
df = df[cols].copy()

# Огноог datetime болгож, координатын алдаатай мөрүүдийг устгах
df['Зөрчил огноо'] = pd.to_datetime(df['Зөрчил огноо'], errors='coerce')
df = df.dropna(subset=['Уртраг', 'Өргөрөг', 'Зөрчил огноо'])

# 2. Цэгийн карт ба багцлах
# Төвийг тооцох (Монгол улсын ойролцоо)
center_lat, center_lon = df['Өргөрөг'].mean(), df['Уртраг'].mean()
m_point = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# MarkerCluster ашиглах
marker_cluster = MarkerCluster(name='Зөрчил цэгүүд').add_to(m_point)
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Өргөрөг'], row['Уртраг']],
        radius=3,
        popup=(f"{row['Зөрчил огноо'].date()}<br>"
               f"{row['Аймгийн нэр']}, {row['Дүүргийн нэр']}<br>"
               f"{row['Авто зам - Замын ангилал']}"),
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(marker_cluster)

# HeatMap ашиглах (давтамжийн халуун цэг)
heat_data = list(zip(df['Өргөрөг'], df['Уртраг']))
HeatMap(heat_data, name='HeatMap').add_to(m_point)

# 3. Хар цэг судлал (Choropleth) — дүүргээр
# Дүүргийн шейпфайл унших (Өөрийн замыг зааж өгөх)
shp_path = 'mongolia_districts.shp'
gdf_districts = gpd.read_file(shp_path)

# Зөрчлийг дүүргээр тоолох
agg = df.groupby('Дүүргийн нэр').size().reset_index(name='count')
# Геодататай нэгдэх
gdf_districts = gdf_districts.merge(agg, left_on='DISTRICT_NAME', right_on='Дүүргийн нэр', how='left')
gdf_districts['count'] = gdf_districts['count'].fillna(0)

# Folium дээр Choropleth үүсгэх
folium.Choropleth(
    geo_data=gdf_districts.__geo_interface__,
    data=gdf_districts,
    columns=['DISTRICT_NAME', 'count'],
    key_on='feature.properties.DISTRICT_NAME',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Зөрчлийн тоо (Дүүрэг)',
    name='Choropleth – Дүүрэг'
).add_to(m_point)

# Газрын зураг дээр давхар давхарга тохируулах
folium.LayerControl(collapsed=False).add_to(m_point)

# HTML файл болгон хадгалах
m_point.save('traffic_violation_geodistribution.html')

print("Географик анализын газрын зураг бэлэн боллоо: traffic_violation_geodistribution.html")


ModuleNotFoundError: No module named 'geopandas'

In [18]:
import asyncio
import platform
import pandas as pd
import pygame
from pygame import gfxdraw

FPS = 60

# Data processing functions
def process_data(raw_data):
    try:
        # Assuming loadFileData provides Excel content as a string; use BytesIO for Pyodide
        from io import BytesIO
        df = pd.read_excel(BytesIO(raw_data.encode()), engine='openpyxl')
        df_clean = df.dropna(subset=['Уртраг', 'Өргөрөг']).copy()
        df_clean['Уртраг'] = pd.to_numeric(df_clean['Уртраг'], errors='coerce')
        df_clean['Өргөрөг'] = pd.to_numeric(df_clean['Өргөрөг'], errors='coerce')
        df_clean = df_clean.dropna(subset=['Уртраг', 'Өргөрөг'])
        df_clean['Зөрчил огноо'] = pd.to_datetime(df_clean['Зөрчил огноо'], errors='coerce')
        return df_clean
    except Exception as e:
        print(f"Error processing data: {e}")
        return pd.DataFrame()

def get_province_counts(df):
    return df['Аймгийн нэр'].value_counts().head(10).sort_values(ascending=False)

def get_road_class_counts(df):
    return df['Авто зам - Замын ангилал'].value_counts()

def get_monthly_counts(df):
    return df.resample('M', on='Зөрчил огноо').size()

# Pygame setup and drawing function
def setup():
    global screen, font, df, province_counts, road_class_counts, monthly_counts
    pygame.init()
    screen = pygame.display.set_mode((1200, 800))
    pygame.display.set_caption("Traffic Violations Visualizations")
    font = pygame.font.SysFont(None, 24)
    raw_data = loadFileData('ЗТО_2020-2024_ашиглах_final.xlsx')
    if not raw_data:
        print("No data loaded")
        return False
    df = process_data(raw_data)
    if df.empty:
        print("Data processing failed")
        return False
    province_counts = get_province_counts(df)
    road_class_counts = get_road_class_counts(df)
    monthly_counts = get_monthly_counts(df[df['Зөрчил огноо'].notna()])
    return True

def draw_map():
    gfxdraw.box(screen, (50, 50, 200, 200), (200, 200, 200))  # Gray placeholder
    text = font.render("Interactive Map (Placeholder)", True, (0, 0, 0))
    screen.blit(text, (60, 60))

def draw_province_bar():
    bar_width = 400
    bar_height = 30
    padding = 10
    max_count = max(province_counts) if province_counts.any() else 1
    y_start = 300
    for i, (province, count) in enumerate(province_counts.items()):
        y = y_start + i * (bar_height + padding)
        bar_length = min((count / max_count) * bar_width, bar_width)  # Cap length
        gfxdraw.box(screen, (500, y, int(bar_length), bar_height), (128, 0, 128))  # Purple
        text = font.render(f"{province[:20]}: {count}", True, (0, 0, 0))  # Truncate long names
        screen.blit(text, (910, y))

def draw_road_class_pie():
    total = road_class_counts.sum()
    if total == 0:
        return
    start_angle = 0
    center = (300, 600)
    radius = 100
    colors = [(128, 0, 128), (150, 50, 150), (100, 0, 100), (170, 100, 170)]
    for i, count in enumerate(road_class_counts):
        angle = (count / total) * 360
        end_angle = start_angle + angle
        gfxdraw.arc(screen, center[0], center[1], radius, int(start_angle), int(end_angle), colors[i % len(colors)])
        text = font.render(f"{road_class_counts.index[i][:15]}: {count}", True, (0, 0, 0))  # Truncate labels
        screen.blit(text, (450, 550 + i * 20))
        start_angle = end_angle

def draw_monthly_line():
    x_offset = 750
    y_offset = 300
    max_count = max(monthly_counts) if monthly_counts.any() else 1
    points = [(x_offset + i * 20, y_offset - min((count / max_count) * 200, 200)) for i, count in enumerate(monthly_counts)]
    if points:
        pygame.draw.lines(screen, (128, 0, 128), False, points, 2)  # Purple line
        for i, (x, y) in enumerate(points):
            pygame.draw.circle(screen, (128, 0, 128), (int(x), int(y)), 3)
            text = font.render(f"{monthly_counts.index[i].strftime('%Y-%m')}: {count}", True, (0, 0, 0))
            screen.blit(text, (x - 50, y - 20))

def update_loop():
    screen.fill((255, 255, 255))  # White background
    draw_map()
    draw_province_bar()
    draw_road_class_pie()
    draw_monthly_line()
    pygame.display.flip()

async def main():
    if not setup():
        return
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        update_loop()
        await asyncio.sleep(1.0 / FPS)
    pygame.quit()

if platform.system() == "Emscripten":
    asyncio.ensure_future(main())
else:
    if __name__ == "__main__":
        asyncio.run(main())

ModuleNotFoundError: No module named 'pygame'

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


# Өгөгдөл унших
df = pd.read_excel("ЗТО_2020-2024_ашиглах_final.xlsx")

# Шаардлагатай багануудыг шалгах
required_columns = [
    'Дүүргийн нэр', 'Аймгийн нэр', 'Зөрчил огноо',
    'Авто зам - Замын ангилал', 'Авто зам - Замын хучилт (Асфальт)',
    'Авто зам - Замын гадаргуу', 'Авто зам - Замын онцлог',
    'Авто зам - Зорчих хэсгийн өргөн', 'Авто зам - Үзэгдэх орчин',
    'Авто зам - Цаг агаар', 'Авто зам - Бусад (Өдөр)', 'Авто зам - Замын хэсэг'
]

# Зөвхөн шаардлагатай багануудыг авч үзэх
df_clean = df[required_columns].copy()


# Дүүргээр зөрчлийн тоо
district_counts = df_clean['Дүүргийн нэр'].value_counts().reset_index()
district_counts.columns = ['Дүүрэг', 'Зөрчлийн тоо']

# Аймгаар зөрчлийн тоо
province_counts = df_clean['Аймгийн нэр'].value_counts().reset_index()
province_counts.columns = ['Аймаг', 'Зөрчлийн тоо']

# Харьцуулалт
plt.figure(figsize=(16, 10))
plt.subplot(2, 1, 1)
sns.barplot(
    x='Зөрчлийн тоо', 
    y='Дүүрэг', 
    data=district_counts.head(20),  # Топ 20 дүүрэг
    palette='Blues_d'
)
plt.title('Топ 20 дүүргээр зөрчлийн тоо')
plt.ylabel('Дүүрэг')

plt.subplot(2, 1, 2)
sns.barplot(
    x='Зөрчлийн тоо', 
    y='Аймаг', 
    data=province_counts,
    palette='Reds_d'
)
plt.title('Аймгаар зөрчлийн тоо')
plt.ylabel('Аймаг')
plt.tight_layout()
plt.savefig('district_province_comparison.png')



# Замын ангилал & дүүргээр
plt.figure(figsize=(16, 12))
sns.countplot(
    y='Дүүргийн нэр',
    hue='Авто зам - Замын ангилал',
    data=df_clean,
    order=df_clean['Дүүргийн нэр'].value_counts().index[:10],  # Топ 10 дүүрэг
    palette='Set2'
)
plt.title('Топ 10 дүүрэг дэх замын ангилалаар зөрчлийн тархалт')
plt.xlabel('Зөрчлийн тоо')
plt.ylabel('Дүүрэг')
plt.legend(title='Замын ангилал', loc='lower right')
plt.tight_layout()
plt.savefig('road_class_by_district.png')

# Замын ангилал & аймгаар
plt.figure(figsize=(16, 8))
sns.countplot(
    y='Аймгийн нэр',
    hue='Авто зам - Замын ангилал',
    data=df_clean,
    palette='Set3'
)
plt.title('Аймгаар замын ангилалаар зөрчлийн тархалт')
plt.xlabel('Зөрчлийн тоо')
plt.ylabel('Аймаг')
plt.legend(title='Замын ангилал', loc='lower right')
plt.tight_layout()
plt.savefig('road_class_by_province.png')


# Замын хучилт (Асфальт) - дүүргээр
plt.figure(figsize=(16, 10))
sns.countplot(
    y='Дүүргийн нэр',
    hue='Авто зам - Замын хучилт (Асфальт)',
    data=df_clean,
    order=df_clean['Дүүргийн нэр'].value_counts().index[:10],
    palette='viridis'
)
plt.title('Топ 10 дүүрэг дэх замын хучилтаар зөрчлийн тархалт')
plt.xlabel('Зөрчлийн тоо')
plt.ylabel('Дүүрэг')
plt.legend(title='Асфальт хучилт', loc='lower right')
plt.tight_layout()
plt.savefig('road_surface_by_district.png')

# Цаг агаарын нөхцөлөөр - аймгаар
plt.figure(figsize=(16, 8))
sns.countplot(
    y='Аймгийн нэр',
    hue='Авто зам - Цаг агаар',
    data=df_clean,
    palette='coolwarm'
)
plt.title('Аймгаар цаг агаарын нөхцөлөөр зөрчлийн тархалт')
plt.xlabel('Зөрчлийн тоо')
plt.ylabel('Аймаг')
plt.legend(title='Цаг агаар', loc='lower right')
plt.tight_layout()
plt.savefig('weather_by_province.png')


# Дүүрэг & замын ангилалаар
pivot_district = pd.pivot_table(
    df_clean,
    index='Дүүргийн нэр',
    columns='Авто зам - Замын ангилал',
    values='Зөрчил огноо',
    aggfunc='count',
    fill_value=0
).loc[district_counts['Дүүрэг'].head(10)]  # Топ 10 дүүрэг

# Аймаг & замын хэсгээр
pivot_province = pd.pivot_table(
    df_clean,
    index='Аймгийн нэр',
    columns='Авто зам - Замын хэсэг',
    values='Зөрчил огноо',
    aggfunc='count',
    fill_value=0
)

# Pivot table-уудыг харах
print("Дүүргээр замын ангилал:")
print(pivot_district)

print("\nАймгаар замын хэсэг:")
print(pivot_province)

# Дээрх бүх графикуудыг файлд хадгалсан
# Нэмэлт: Корреляцийн матриц
corr_columns = [
    'Авто зам - Зорчих хэсгийн өргөн',
    'Авто зам - Үзэгдэх орчин',
    'Авто зам - Цаг агаар'
]

# Тоон утга руу хөрвүүлэх (хэрэгтэй


KeyError: "['Авто зам - Замын хучилт (Асфальт)', 'Авто зам - Бусад (Өдөр)'] not in index"